## 1. 검색 성능 평가: 왜 해야 하는가?

- **'감'이 아닌 '데이터'로 말하기:** 구축한 검색 시스템(Retriever)이 얼마나 잘 동작하는지 정량적으로 평가하는 것은 매우 중요함. '왠지 잘 되는 것 같다'는 느낌을 넘어, 시스템의 강점과 약점을 객관적인 숫자로 파악할 수 있음.
- **개선 방향 설정:** 평가 지표를 통해 어떤 종류의 질문에 취약한지, 어떤 파라미터를 튜닝해야 할지 등 개선 방향을 명확히 설정할 수 있음.

### 평가의 장단점
- **장점:**
  - **객관성 확보:** 여러 Retriever나 설정 변경에 따른 성능을 공정하게 비교 가능함.
  - **병목 현상 진단:** RAG 파이프라인에서 성능 저하의 원인이 Retriever인지, LLM의 답변 생성 능력인지 구분하는 데 도움이 됨.
  - **점진적 개선:** 평가-개선 사이클을 반복하며 시스템을 체계적으로 발전시킬 수 있음.
- **단점:**
  - **초기 비용:** 양질의 평가 데이터셋을 구축하는 데 시간과 노력이 소요됨.
  - **지표의 한계:** 특정 지표가 높다고 해서 사용자 만족도가 항상 비례하는 것은 아님. 정성적 평가를 병행하는 것이 좋음.


### 1.1 테스트 데이터 준비

- 좋은 평가를 위해서는 양질의 테스트 데이터셋이 필요함. 
- 여기서는 기존 문서들을 기반으로 질문-답변(QA) 쌍을 합성하고, 이를 검토/수정하여 사용함.

## 2. 테스트 데이터 준비: GIGO(Garbage In, Garbage Out)

- 좋은 평가는 좋은 테스트 데이터셋에서 시작됨. 쓰레기 같은 데이터로 평가하면 쓰레기 같은 결론만 나옴.
- 여기서는 기존 문서들을 기반으로 **(1) 데이터 정제 → (2) QA 쌍 합성 → (3) QA 검수**의 3단계로 고품질 평가셋을 구축하는 과정 제시.

### 2.1. 평가용 문서 데이터 정제

- `korean_docs`를 평가용으로 좀 더 세밀하게 가공하는 단계임.
  - **고유 ID 부여:** 각 문서 조각(청크)에 고유 `doc_id` 메타데이터를 추가하여 나중에 정답을 정확히 식별할 수 있게 함.
  - **컨텍스트 명시:** 문서 내용 끝에 "이 문서는 'Tesla'에 대한 정보입니다."와 같이 출처를 명시함. 이는 LLM이 QA를 생성할 때 어떤 기업에 대한 내용인지 명확히 인지하여 더 좋은 품질의 QA를 생성하도록 유도하는 장치임.
- JSONL (JSON Lines) 형식으로 저장하여 대용량 데이터를 효율적으로 처리하고 재사용성을 높임.


#### 장점과 단점
- **장점:** 
  - **평가의 신뢰성 향상:** `doc_id`로 정답 문서를 명확히 추적할 수 있어 평가의 정확도가 올라감.
  - **QA 품질 향상:** LLM에 명시적인 컨텍스트를 제공하여, 더 사실에 기반하고 맥락에 맞는 QA 쌍을 생성하게 함.
  - **재현성 확보:** 정제된 데이터를 파일로 저장해두면, 언제든 동일한 조건으로 평가를 재현할 수 있음.
- **단점:** 
  - **수동 공수 발생:** 초기 데이터셋을 구축하고 정제 규칙을 만드는 데 많은 노력이 듬.
  - **전처리 복잡도 증가:** 단순 텍스트 로딩보다 파이프라인이 약간 더 복잡해짐.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader
from glob import glob
import os 

In [3]:
def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        print(f"로딩 중: {text_file}")
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

In [4]:
korean_txt_files = glob(os.path.join('../data', '*_KR.txt'))
if not korean_txt_files:
    print("'data' 폴더에 '*_KR.txt' 파일이 없습니다. 예시 데이터를 생성하거나 경로를 확인하세요.")
    korean_data = [] # 빈 리스트로 초기화
else:
    korean_data = load_text_files(korean_txt_files)

print(f"\n로드된 전체 문서 수: {len(korean_data)}")
if korean_data:
    print(f"첫 번째 문서 내용 일부: {korean_data[0].page_content[:100]}...")


로딩 중: ../data\Rivian_KR.txt
로딩 중: ../data\Tesla_KR.txt

로드된 전체 문서 수: 2
첫 번째 문서 내용 일부: 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" S...


In [5]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 문장 구분자(정규식)를 사용하여 텍스트 분할기 생성
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,          # 토큰 수 계산 기준
    separator=r"[.!?]\s+",     # 문장 구분자: 마침표, 느낌표, 물음표 뒤 공백
    chunk_size=100,             # 청크 최대 토큰 수 (토크나이저 기준)
    chunk_overlap=20,            # 청크 간 중복 토큰 수
    is_separator_regex=True,    # separator가 정규식임을 명시
    keep_separator=True,        # 구분자 유지 여부
)

In [6]:
korean_docs = []
if korean_data: # 로드된 데이터가 있을 경우에만 분할 수행
    korean_docs = text_splitter.split_documents(korean_data)

print(f"\n분할된 한국어 문서 수: {len(korean_docs)}")
if korean_docs:
    print(f"첫 번째 분할 문서 내용: {korean_docs[0].page_content}")
    print(f"첫 번째 분할 문서 메타데이터: {korean_docs[0].metadata}")


분할된 한국어 문서 수: 5
첫 번째 분할 문서 내용: 리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임
첫 번째 분할 문서 메타데이터: {'source': '../data\\Rivian_KR.txt'}


In [7]:
import json 
final_docs_for_eval = []
if korean_docs: 
    for i, doc in enumerate(korean_docs):
        new_doc = doc.copy() 
        new_doc.metadata['doc_id'] = f'{i}'
        source_filename = os.path.basename(new_doc.metadata.get('source', ''))
        corp_name = source_filename.split('_')[0] if '_' in source_filename else source_filename.replace('.txt', '')
        new_doc.page_content = f"{new_doc.page_content}\n\n(참고: 이 문서는 '{corp_name}'에 대한 정보를 담고 있습니다.)"
        final_docs_for_eval.append(new_doc)

    print(f"평가용으로 정제된 문서 수: {len(final_docs_for_eval)}")
    if final_docs_for_eval:
        print("\n첫 번째 정제 문서 예시:")
        print(final_docs_for_eval[0].page_content)
        print("-" * 30)
        print(final_docs_for_eval[0].metadata)
        print("=" * 30)

    with open('../data/final_docs_for_eval.jsonl', 'w', encoding='utf-8') as f:
        for doc_item in final_docs_for_eval:
            f.write(json.dumps({'page_content': doc_item.page_content, 'metadata': doc_item.metadata}) + '\n')
    print("\n평가용 문서가 './data/final_docs_for_eval.jsonl' 파일로 저장되었습니다.")

else:
    print("korean_docs가 없어 평가용 문서를 정제할 수 없습니다.")

평가용으로 정제된 문서 수: 5

첫 번째 정제 문서 예시:
리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임

(참고: 이 문서는 'Rivian'에 대한 정보를 담고 있습니다.)
------------------------------
{'source': '../data\\Rivian_KR.txt', 'doc_id': '0'}

평가용 문서가 './data/final_docs_for_eval.jsonl' 파일로 저장되었습니다.


C:\Users\USER\AppData\Local\Temp\ipykernel_28828\42143547.py:5: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_doc = doc.copy()


`(2) Question-Answer (QA) 합성`

- 정제된 문서(`final_docs_for_eval`)의 각 청크(문서)를 컨텍스트로 하여, LLM(여기서는 OpenAI GPT 모델)을 사용해 질문과 답변 쌍을 생성함.
- **`PydanticOutputParser` 활용:** LLM의 출력이 우리가 원하는 `QAPair`, `QASet` 같은 구조화된 객체 형태로 나오도록 강제함. 이렇게 하면 후속 처리가 매우 깔끔해짐.
- **프롬프트 엔지니어링:** '검색 엔진 사용자처럼 질문해줘', '답변에 질문의 핵심 내용을 포함해줘' 와 같이 구체적인 지시를 통해 원하는 스타일과 품질의 QA를 얻어내는 것이 핵심임.

#### 장점과 단점
- **장점:**
  - **압도적인 속도와 효율성:** 사람이 직접 수백, 수천 개의 QA를 만드는 것에 비해 훨씬 빠르고 저렴하게 대규모 평가 데이터셋을 구축할 수 있음.
  - **다양성 확보:** LLM은 사람이 생각하지 못한 다양한 관점의 질문을 생성해낼 수 있음.
- **단점:**
  - **API 비용 발생:** LLM API를 호출하므로 비용이 발생함. (특히 문서가 많을 경우)
  - **품질 불확실성 (환각 현상):** LLM이 컨텍스트에 없는 내용을 지어내거나(Hallucination), 질문의 의도를 잘못 파악하는 경우가 발생할 수 있음. **따라서 후속 검수 과정이 필수적임.**
  - **프롬프트 민감도:** 프롬프트의 미세한 차이에도 결과물의 품질이 크게 달라질 수 있어, 최적의 프롬프트를 찾는 데 시간이 걸릴 수 있음.

#### 🔑 **개인적 생각**
> 처음부터 모든 문서에 대해 QA 생성을 돌리지 말 것. 5~10개 정도의 소수 문서로 먼저 테스트하며 프롬프트와 LLM 모델(e.g., gpt-4o-mini vs gpt-4o)을 튜닝하는 것이 비용과 시간을 아끼는 길이라고 생각. `gpt-4o-mini`는 가성비가 좋은 선택지가 될 수 있음.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import pandas as pd
from tqdm.auto import tqdm # 진행률 표시를 위한 라이브러리

# Pydantic 모델: LLM의 출력 구조를 정의함
# QAPair: 개별 질문-답변 쌍
class QAPair(BaseModel):
    question: str = Field(description="주어진 컨텍스트를 기반으로 생성된, 사실에 입각한 질문 (한국어)")
    answer: str = Field(description="생성된 질문에 대한 명확하고 간결한 답변 (한국어, 컨텍스트 내의 정보만 사용)")

# QASet: QAPair의 리스트를 담는 컨테이너. LLM이 여러 개의 QA를 생성할 때 사용
class QASet(BaseModel):
    qa_pairs: List[QAPair] = Field(description="질문-답변(QAPair) 쌍의 리스트")

# QA 생성을 위한 프롬프트 템플릿
QA_GENERATION_TEMPLATE_KOREAN = """
당신은 주어진 컨텍스트를 기반으로 사실에 입각한 질문-답변 쌍을 생성하는 전문 AI입니다.
오직 제공된 컨텍스트 내의 정보만을 사용하여, 특정하고 간결한 사실로 답변할 수 있는 질문을 {num_questions_per_chunk}개 만드세요.

## 지침:
1. 질문은 실제 사용자가 검색 엔진에 입력할 법한 자연스러운 스타일로 작성해주세요.
2. "제시된 구절에 따르면", "컨텍스트에 따르면" 같은 표현은 절대 사용하지 마세요.
3. 답변은 질문의 핵심 내용을 포함하여, 완전한 문장 형태로 명확하게 제공해주세요.
4. 컨텍스트에 정보가 부족하여 질문을 만들기 어렵다면, 빈 리스트를 반환하세요.

---------------------------------------------------------
## 출력 형식:
{format_instructions}
---------------------------------------------------------
## 컨텍스트:
{context}
"""

# QA 생성에 사용할 LLM 정의 (gpt-4o-mini는 비용 효율적임)
qa_llm = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=1000,
    temperature=0.1, # 사실 기반 생성이므로 낮은 온도로 설정
)

# PydanticOutputParser: LLM 출력을 QASet 객체로 파싱
pydantic_parser_qa = PydanticOutputParser(pydantic_object=QASet)

# 프롬프트 템플릿 생성
qa_generation_prompt = ChatPromptTemplate.from_template(
    template=QA_GENERATION_TEMPLATE_KOREAN,
    partial_variables={"format_instructions": pydantic_parser_qa.get_format_instructions()}
)

# 체인(Chain) 구성: Prompt -> LLM -> Parser
qa_generation_chain = qa_generation_prompt | qa_llm | pydantic_parser_qa

# QA 생성 테스트 (첫 번째 정제 문서 사용)
if final_docs_for_eval:
    test_context_for_qa = final_docs_for_eval[0].page_content
    print(f"QA 생성 테스트용 컨텍스트:\n{test_context_for_qa}\n")
    
    try:
        qa_set_example = qa_generation_chain.invoke({
            "context": test_context_for_qa,
            "num_questions_per_chunk": 1 # 테스트로 1개만 생성
        })
        print("생성된 QA 쌍 예시:")
        for qa_pair in qa_set_example.qa_pairs:
            print(f"  질문: {qa_pair.question}")
            print(f"  답변: {qa_pair.answer}")
    except Exception as e:
        print(f"테스트 실행 중 오류 발생: {e}")
        print("OpenAI API 키가 올바르게 설정되었는지 확인하세요.")
else:
    print("정제된 평가용 문서가 없어 QA 생성 테스트를 스킵함.")

QA 생성 테스트용 컨텍스트:
리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동차 제조업체이자 자동차 기술 회사임.
2009년에 로버트 "RJ" 스캐린지(Robert "RJ" Scaringe)에 의해 설립되었음. 본사는 캘리포니아주 어바인에 위치해 있음.
리비안의 주력 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV임

(참고: 이 문서는 'Rivian'에 대한 정보를 담고 있습니다.)

생성된 QA 쌍 예시:
  질문: 리비안 오토모티브는 언제 설립되었나요?
  답변: 리비안 오토모티브는 2009년에 설립되었습니다.


In [9]:
# 전체 평가용 문서에 대해 QA 생성
NUM_QUESTIONS_PER_CHUNK = 2 # 각 문서 청크당 생성할 QA 개수
generated_qa_outputs = []

if final_docs_for_eval :
    print(f"{len(final_docs_for_eval)}개의 문서에 대해 QA 생성을 시작합니다...")
    for i, doc_item in enumerate(final_docs_for_eval):
        print(f"{i+1}/{len(final_docs_for_eval)}번째 문서 처리 중...", end=' ')
        try:
            qa_set_generated = qa_generation_chain.invoke({
                "context": doc_item.page_content,
                "num_questions_per_chunk": NUM_QUESTIONS_PER_CHUNK
            })
            for qa_pair in qa_set_generated.qa_pairs:
                generated_qa_outputs.append({
                    'context': [doc_item.page_content], # 정답 컨텍스트 (리스트 형태)
                    'source': [doc_item.metadata.get('source', '')], # 출처 (리스트 형태)
                    'doc_id': [doc_item.metadata.get('doc_id', '')], # 문서 ID (리스트 형태)
                    'question': qa_pair.question,
                    'answer': qa_pair.answer
                })
            print("성공")
        except Exception as e:
            print(f"실패: {e}")
            continue # 오류 발생 시 다음 문서로
    
    df_generated_qa_test = pd.DataFrame(generated_qa_outputs)
    print(f"\n총 {df_generated_qa_test.shape[0]}개의 QA 쌍이 생성되었습니다.")
    df_generated_qa_test.to_excel("../data/generated_qa_test.xlsx", index=False)
else:
    print("QA 합성은 실제 실행되지 않았습니다. (final_docs_for_eval이 없거나, 실행 플래그가 False임)")

5개의 문서에 대해 QA 생성을 시작합니다...
1/5번째 문서 처리 중... 성공
2/5번째 문서 처리 중... 성공
3/5번째 문서 처리 중... 성공
4/5번째 문서 처리 중... 성공
5/5번째 문서 처리 중... 성공

총 10개의 QA 쌍이 생성되었습니다.


In [10]:
df_generated_qa_test

,context,source,doc_id,question,answer
0,"[리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동...",[../data\Rivian_KR.txt],[0],리비안 오토모티브는 언제 설립되었나요?,리비안 오토모티브는 2009년에 설립되었습니다.
1,"[리비안 오토모티브(Rivian Automotive, Inc.)는 미국의 전기 자동...",[../data\Rivian_KR.txt],[0],리비안의 본사는 어디에 위치하고 있나요?,리비안의 본사는 캘리포니아주 어바인에 위치해 있습니다.
2,"[. 이들 차량은 ""스케이트보드"" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 ...",[../data\Rivian_KR.txt],[1],리비안은 어떤 플랫폼을 기반으로 차량을 제작하나요?,리비안은 '스케이트보드' 플랫폼을 기반으로 차량을 제작합니다.
3,"[. 이들 차량은 ""스케이트보드"" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 ...",[../data\Rivian_KR.txt],[1],리비안은 어떤 기업으로부터 투자를 받았나요?,리비안은 아마존과 포드 등 주요 기업으로부터 투자를 받았습니다.
4,"[테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.\n2...",[../data\Tesla_KR.txt],[2],테슬라는 언제 설립되었나요?,테슬라는 2003년에 설립되었습니다.
5,"[테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.\n2...",[../data\Tesla_KR.txt],[2],테슬라의 CEO는 누구인가요?,테슬라의 CEO는 일론 머스크입니다.
6,[. 본사는 텍사스주 오스틴에 있음.\n테슬라의 대표적인 전기차 모델로는 모델 S ...,[../data\Tesla_KR.txt],[3],테슬라의 본사는 어디에 있나요?,테슬라의 본사는 텍사스주 오스틴에 있습니다.
7,[. 본사는 텍사스주 오스틴에 있음.\n테슬라의 대표적인 전기차 모델로는 모델 S ...,[../data\Tesla_KR.txt],[3],테슬라의 대표적인 전기차 모델은 무엇이 있나요?,"테슬라의 대표적인 전기차 모델로는 모델 S, 모델 3, 모델 X, 모델 Y, 그리고..."
8,"[.\n또한, 테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Power...",[../data\Tesla_KR.txt],[4],테슬라는 어떤 에너지 관련 제품을 생산하나요?,"테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Powerwall), 대..."
9,"[.\n또한, 테슬라는 태양광 패널, 가정용 에너지 저장 시스템인 파워월(Power...",[../data\Tesla_KR.txt],[4],테슬라는 어떤 기술 개발에 많은 투자를 하고 있나요?,테슬라는 자율 주행 기술 개발에 많은 투자를 하고 있습니다.


### 2.3. 테스트 데이터 검토 및 수정

- **가장 중요하지만 가장 간과하기 쉬운 단계.** LLM이 생성한 QA 데이터는 완벽하지 않으므로, 사람이 직접 검토하고 수정하는 과정이 반드시 필요함.
- Excel 등으로 저장된 `generated_qa_test.xlsx` 파일을 열어 아래 사항들을 점검하고 수정함:
  - 질문이 어색하거나 모호하지 않은가?
  - 답변이 질문에 대해 정확하게 대답하고 있는가?
  - 답변이 컨텍스트(ground_truth_context)에 실제로 있는 내용인가? (환각은 아닌가?)
  - 더 좋은 질문이나 답변으로 개선할 수 있는가?


#### 장점과 단점
- **장점:**
  - **평가 신뢰도 극대화:** 잘못된 데이터로 평가하여 잘못된 결론에 도달하는 것을 방지함. 이 과정을 거친 데이터셋은 시스템의 실제 성능을 가장 잘 반영함.
  - **'어려운' 질문 발굴:** 시스템이 잘 답변하지 못하는 까다로운 질문(Hard negatives)들을 발견하고 평가셋에 추가하여 견고성을 테스트할 수 있음.
- **단점:**
  - **명백한 단점: 시간과 인력:** 데이터의 양이 많을수록 많은 시간과 노력이 필요한 수동 작업임. 

## 3. 정보 검색(IR) 평가지표

- 이제 잘 준비된 평가 데이터셋으로 Retriever의 성능을 측정할 차례임.
- 정보 검색(IR) 분야에서 오랫동안 사용되어 온 표준 지표들을 사용하여, "Retriever가 사용자의 질문(query)에 대해 얼마나 관련 있는 문서를, 얼마나 높은 순위로 가져오는지"를 평가함.
- 주요 지표: **Hit Rate, MRR, mAP, NDCG** 등. 각 지표는 서로 다른 관점에서 성능을 측정하므로, 여러 지표를 함께 보는 것이 바람직함.

In [11]:
from langchain_core.documents import Document
import numpy as np

# 각 쿼리에 대한 정답 문서
actual_docs_sample = [
    [Document(metadata={'doc_id': 'doc1'}, page_content='내용1')],
    [Document(metadata={'doc_id': 'doc2'}, page_content='내용2'),
     Document(metadata={'doc_id': 'doc5'}, page_content='내용5')]
]

# 각 쿼리에 대한 검색기가 예측한 문서
predicted_docs_sample = [
    [Document(metadata={'doc_id': 'doc1'}, page_content='내용1'),
     Document(metadata={'doc_id': 'doc3'}, page_content='내용3')],
    [Document(metadata={'doc_id': 'doc4'}, page_content='내용4'),
     Document(metadata={'doc_id': 'doc1'}, page_content='내용1'),
     Document(metadata={'doc_id': 'doc5'}, page_content='내용5'),
     Document(metadata={'doc_id': 'doc2'}, page_content='내용2')]
]

print("샘플 정답 문서 및 예측 문서 준비 완료.")

샘플 정답 문서 및 예측 문서 준비 완료.


### 3.1. Hit Rate @k (적중률)

- **개념:** "상위 k개 결과 안에 정답이 하나라도 있는가?" (Yes or No)
  - 각 쿼리에 대해 검색된 상위 `k`개의 문서 중에 실제 정답 문서가 **하나라도 포함**되어 있으면 1점 (Hit), 아니면 0점.
  - 전체 쿼리에 대한 점수의 평균을 냄.
- **장점:** 가장 간단하고 직관적임. Retriever가 최소한의 역할을 하는지 빠르게 파악 가능.
- **단점:** 정답 문서의 순위(1등인지 k등인지)나 여러 정답 중 몇 개를 맞췄는지는 전혀 고려하지 않음.
- **언제 사용할까?** "일단 관련 문서를 찾기만 하면 된다"는 최소한의 성능을 측정하고 싶을 때 유용함.

In [12]:
def hit_rate_at_k(actual_doc_ids_list, predicted_docs_list, k=4):
    hits = 0
    for actual_docs, predicted in zip(actual_doc_ids_list, predicted_docs_list):
        predicted_ids_at_k = {p.metadata['doc_id'] for p in predicted[:k]}
        actual_ids = {d.metadata['doc_id'] for d in actual_docs}
        if any(doc_id in predicted_ids_at_k for doc_id in actual_ids):
            hits += 1
    return hits / len(actual_doc_ids_list)


hr_score = hit_rate_at_k(actual_docs_sample, predicted_docs_sample, k=4)
print(f"Hit Rate @4: {hr_score:.4f}")

Hit Rate @4: 1.0000


### 3.2. MRR @k (Mean Reciprocal Rank)

- **개념:** "첫 정답을 얼마나 빨리(높은 순위로) 찾았는가?"
  - 각 쿼리에 대해 **첫 번째로 발견된 정답 문서**의 순위(rank)의 역수(`1/rank`)를 계산.
  - 이 값들의 평균을 냄. 만약 상위 `k`개 안에 정답이 없으면 0점.
- **장점:** 정답의 순위를 고려함. 사용자가 원하는 결과를 얼마나 빨리 찾는지 평가하기 좋음.
- **단점:** 첫 번째 정답만 신경 씀. 그 뒤에 다른 정답들이 더 있어도 무시함.
- **언제 사용할까?** 사용자가 단 하나의 정답을 찾는 '사실 확인형' 질문(e.g., "테슬라의 CEO는 누구?")에 대한 성능 평가에 적합함.

---

In [13]:
def mrr_at_k(actual_doc_ids_list, predicted_docs_list, k=4):
    reciprocal_ranks = []
    for actual_docs, predicted in zip(actual_doc_ids_list, predicted_docs_list):
        actual_ids = {d.metadata['doc_id'] for d in actual_docs}
        rank = 0
        for i, p in enumerate(predicted[:k]):
            if p.metadata['doc_id'] in actual_ids:
                rank = i + 1
                break
        reciprocal_ranks.append(1 / rank if rank > 0 else 0)
    return sum(reciprocal_ranks) / len(reciprocal_ranks)

mrr_score = mrr_at_k(actual_docs_sample, predicted_docs_sample, k=4)
print(f"MRR @4: {mrr_score:.4f}")


MRR @4: 0.6667


### 3.3. mAP @k (Mean Average Precision)

- **개념:** "관련된 문서를 얼마나 많이, 그리고 얼마나 높은 순위로 가져왔는가?"
  - 정답이 여러 개일 때, 순서와 재현율(Recall)을 함께 고려하는 지표. 
  - 각 쿼리에 대한 AP(Average Precision)를 구하고, 이들의 평균을 냄.
  - AP는 상위 `k`개 결과 내에서 각 **정답 문서가 나올 때마다의 정밀도(Precision) 값**들의 평균임.
- **장점:** 검색 결과의 순서와 정확도를 모두 고려함. 여러 정답이 있는 경우에 특히 유용함.
- **단점:** 개념이 다른 지표보다 조금 복잡함.
- **언제 사용할까?** 사용자가 하나의 질문으로 여러 관련 정보를 얻고 싶어하는 '탐색형' 질문(e.g., "테슬라 모델 Y의 장점은?")에 대한 성능 평가에 적합함.

---

In [14]:
def precision_at_i(i, actual_ids, predicted_docs):
    predicted_ids_at_i = {p.metadata['doc_id'] for p in predicted_docs[:i]}
    num_correct = len(predicted_ids_at_i.intersection(actual_ids))
    return num_correct / i

def ap_at_k(actual_ids, predicted_docs, k=4):
    if not actual_ids:
        return 0.0
    precisions = []
    for i, p in enumerate(predicted_docs[:k]):
        if p.metadata['doc_id'] in actual_ids:
            precisions.append(precision_at_i(i + 1, actual_ids, predicted_docs))
    if not precisions:
        return 0.0
    return sum(precisions) / len(actual_ids)

def map_at_k(actual_doc_ids_list, predicted_docs_list, k=4):
    aps = [ap_at_k(actual, pred, k) for actual, pred in zip(actual_doc_ids_list, predicted_docs_list)]
    return sum(aps) / len(aps)

map_score = map_at_k(actual_docs_sample, predicted_docs_sample, k=4)
print(f"mAP @4: {map_score:.4f}")

mAP @4: 0.0000


### 3.4. NDCG @k (Normalized Discounted Cumulative Gain)

- **개념:** "가장 이상적인 검색 결과 대비 현재 결과는 얼마나 좋은가?"
  - 검색 결과의 순서와 문서의 **관련성 등급**을 모두 고려하여 평가하는 가장 정교한 지표.
  - **Discounted Cumulative Gain (DCG):** 순위가 낮을수록(log(rank+1)) 패널티를 주어 점수를 합산. 관련성이 높은 문서가 상위에 있을수록 DCG가 높음.
  - **Normalized (N):** DCG 점수를 이상적인 순서일 때의 DCG(IDCG, Ideal DCG)로 나누어 0~1 사이 값으로 정규화함. 이를 통해 쿼리마다 다른 정답 개수나 관련도 분포에 상관없이 공정한 비교가 가능해짐.
- **장점:** 관련성을 이진(0/1)이 아닌 다단계(e.g., 매우 관련 높음=3, 관련 있음=2, 약간 관련=1)로 설정할 수 있어, 문서의 중요도를 세밀하게 반영 가능. 가장 정교한 순위 평가 지표로 알려져 있음.
- **단점:** 개념이 복잡하고, 문서별 관련성 등급을 매기는 추가적인 작업이 필요할 수 있음. (여기서는 정답이면 1, 아니면 0으로 단순화)
- **언제 사용할까?** 검색 결과의 순위가 매우 중요하고, 문서마다 관련성의 정도가 다를 때 가장 강력한 평가 도구가 됨.

---

In [15]:
def dcg_at_k(relevance_scores, k=4):
    scores = np.asfarray(relevance_scores)[:k]
    if scores.size:
        return np.sum(scores / np.log2(np.arange(2, scores.size + 2)))
    return 0.0

def ndcg_at_k(actual_ids, predicted_docs, k=4):
    relevance_scores = [1 if p.metadata['doc_id'] in actual_ids else 0 for p in predicted_docs]
    ideal_scores = sorted(relevance_scores, reverse=True)
    actual_dcg = dcg_at_k(relevance_scores, k)
    ideal_dcg = dcg_at_k(ideal_scores, k)
    if not ideal_dcg:
        return 0.0
    return actual_dcg / ideal_dcg

def mean_ndcg_at_k(actual_doc_ids_list, predicted_docs_list, k=4):
    ndcgs = [ndcg_at_k(actual, pred, k) for actual, pred in zip(actual_doc_ids_list, predicted_docs_list)]
    return np.mean(ndcgs)

ndcg_score = mean_ndcg_at_k(actual_docs_sample, predicted_docs_sample, k=4)
print(f"Mean NDCG @4: {ndcg_score:.4f}")

Mean NDCG @4: 0.0000
